In [1]:
import numpy as np
import logging
import sys
from matplotlib import pyplot as plt

%matplotlib inline
sys.path.append("../")

import torch
from torchvision.datasets import MNIST
from torchvision import transforms

from aef.models.autoencoder import LinearAutoencoder, ConvolutionalAutoencoder, DenseAutoencoder
from aef.trainer import AutoencoderTrainer
from aef.losses import nll, mse

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)



## Data

In [2]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist = MNIST('./data', download=True, transform=img_transform)

## Train autoencoder

In [3]:
ae = DenseAutoencoder(latent_dim=2)

In [4]:
trainer = AutoencoderTrainer(ae, output_filename="../figures/training/dense/dense_phase1")
trainer.train(
    dataset=mnist,
    loss_functions=[mse],
    loss_labels=["MSE"],
    batch_size=256,
    epochs=10,
    verbose="all",
    initial_lr=1.e-3,
    final_lr=1.e-5
)

11:05 aef.trainer          INFO    Epoch   1: train loss  0.30131 (MSE:  0.301)
11:05 aef.trainer          INFO               val. loss   0.23708 (MSE:  0.237)
11:05 aef.trainer          INFO    Epoch   2: train loss  0.22729 (MSE:  0.227)
11:05 aef.trainer          INFO               val. loss   0.22144 (MSE:  0.221)
11:05 aef.trainer          INFO    Epoch   3: train loss  0.21893 (MSE:  0.219)
11:05 aef.trainer          INFO               val. loss   0.21773 (MSE:  0.218)
11:06 aef.trainer          INFO    Epoch   4: train loss  0.21648 (MSE:  0.216)
11:06 aef.trainer          INFO               val. loss   0.21633 (MSE:  0.216)
11:06 aef.trainer          INFO    Epoch   5: train loss  0.21543 (MSE:  0.215)
11:06 aef.trainer          INFO               val. loss   0.21552 (MSE:  0.216)
11:06 aef.trainer          INFO    Epoch   6: train loss  0.21489 (MSE:  0.215)
11:06 aef.trainer          INFO               val. loss   0.21503 (MSE:  0.215)
11:07 aef.trainer          INFO    Epoch

(array([0.30131253, 0.22728518, 0.21892699, 0.21647622, 0.21542842,
        0.21488717, 0.21460149, 0.2144268 , 0.21430679, 0.21423635]),
 array([0.23708012, 0.22144431, 0.21772506, 0.21632947, 0.21552046,
        0.2150287 , 0.21488312, 0.2148284 , 0.21475825, 0.21467779]))

In [5]:
trainer = AutoencoderFlowTrainer(ae, output_filename="../figures/training/dense/dense_phase2")
trainer.train(
    dataset=mnist,
    loss_functions=[nll],
    loss_labels=["NLL"],
    batch_size=256,
    epochs=10,
    verbose="all",
    initial_lr=1.e-3,
    final_lr=1.e-5,
    parameters=ae.flow.parameters()
)

NameError: name 'AutoencoderFlowTrainer' is not defined

## Visualize latent space

In [ ]:
x = torch.cat([mnist[i][0].unsqueeze(0) for i in range(1000)], dim=0)
y = np.asarray([mnist[i][1] for i in range(1000)])

In [ ]:
h, u = ae.latent(x)
x_out = ae.decoder(h)
h = h.detach().numpy().reshape((-1,2))
u = u.detach().numpy().reshape((-1,2))

In [ ]:
fig = plt.figure(figsize=(10,5))

ax = plt.subplot(1,2,1)
for i in range(10):
    plt.scatter(h[y==i][:,0], h[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()
    
ax = plt.subplot(1,2,2)
for i in range(10):
    plt.scatter(u[y==i][:,0], u[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()

plt.tight_layout()
plt.show()


## Visualize reconstruction

In [ ]:
x = torch.cat([mnist[i][0].unsqueeze(0) for i in range(1000)], dim=0)
y = np.asarray([mnist[i][1] for i in range(1000)])

In [ ]:
h = ae.encoder(x)
x_out = ae.decoder(h)
h = h.detach().numpy()
x_out = x_out.detach().numpy()

In [ ]:
fig = plt.figure(figsize=(12,12))

for i in range(18):
    ax = plt.subplot(6, 6, 2*i + 1)
    plt.imshow(x[i].reshape((28,28)))
    ax = plt.subplot(6, 6, 2*i + 2)
    plt.imshow(x_out[i].reshape((28,28)))
        
plt.tight_layout()
plt.show()